In [39]:
from itertools import product

import numpy as np
from scipy.linalg import eigvals
from tangelo import SecondQuantizedMolecule
from tangelo.linq import Circuit, Gate, get_backend
from tangelo.toolboxes.ansatz_generator.ansatz_utils import (
    trotterize,
)
from tangelo.toolboxes.operators import QubitOperator, count_qubits
from tangelo.toolboxes.qubit_mappings.mapping_transform import (
    fermion_to_qubit_mapping as f2q_mapping,
)
from tangelo.toolboxes.qubit_mappings.statevector_mapping import (
    do_scbk_transform,
    vector_to_circuit,
)

from tangelo.algorithms.variational import VQESolver, BuiltInAnsatze
from tangelo.algorithms.classical import FCISolver, CCSDSolver
from tangelo.algorithms.variational import SA_VQESolver

import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem


In [56]:
class HamiltonianSimulator:
    def __init__(self, mol: SecondQuantizedMolecule, mapping="jw"):
        ############################################################################
        # Hamiltonian simulation using Trotter-Suzuki and MRSQK                    #
        ############################################################################
        # Number of Krylov vectors
        self.n_krylov = 4

        # Qubit Mapping
        self.mapping = mapping
        # mapping = 'BK'
        # mapping = 'scBK'

        # Molecule
        self.mol = mol

        # Qubit operator for LiH
        self.qu_op = f2q_mapping(
            mol.fermionic_hamiltonian,
            mapping,
            mol.n_active_sos,
            mol.n_active_electrons,
            up_then_down=False,
            spin=mol.spin,
        )
        self.backend = get_backend()

        self.c_q = count_qubits(self.qu_op)
        print(f"Qubit count: {self.c_q+1}, control qubit: {self.c_q}")

        self.zeroone = QubitOperator(f"X{self.c_q}", 1) + QubitOperator(
            f"Y{self.c_q}", 1j
        )

        # Generate multiple controlled-reference states.
        self.reference_states: list[Circuit] = list()
        reference_vecs = [[1, 1, 0, 0], [1, 0, 0, 1]]
        for vec in reference_vecs:
            circ = vector_to_circuit(vec)
            gates = [
                Gate("C" + gate.name, target=gate.target, control=self.c_q)
                for gate in circ
            ]
            self.reference_states += [Circuit(gates)]

        ############################################################################
        # QChem simulation using Pennylane                                         #
        ############################################################################
        elements = [e[0] for e in mol.xyz]
        coords = np.array([np.array(e[1]) for e in mol.xyz])
        self.qchem_mol = qchem.Molecule(elements, coords, charge=mol.q)
        # mol.n_electrons
        # electrons = 4
        self.qchem_hamiltonian, self.qchem_qubits = qchem.molecular_hamiltonian(
            self.qchem_mol
        )

        hf = qchem.hf_state(mol.n_electrons, self.qchem_qubits)
        singles, doubles = qchem.excitations(electrons=mol.n_electrons, orbitals=self.qchem_qubits)
        num_theta = len(singles) + len(doubles)

        device = qml.device("lightning.qubit", wires=self.qchem_qubits)

        def circuit_VQE(theta, wires):
            qml.AllSinglesDoubles(
                weights=theta, wires=wires, hf_state=hf, singles=singles, doubles=doubles
            )
        
        @qml.qnode(device, interface="autograd")
        def cost_fn(theta):
            circuit_VQE(theta, range(self.qchem_qubits))
            return qml.expval(self.qchem_hamiltonian)

        stepsize = 0.4
        max_iterations = 50
        opt = qml.GradientDescentOptimizer(stepsize=stepsize)
        theta = np.zeros(num_theta, requires_grad=True)

        for n in range(max_iterations):
            theta, prev_energy = opt.step_and_cost(cost_fn, theta)

        # energy_VQE = cost_fn(theta)
        # theta_opt = theta

        @qml.qnode(device)
        def qdrift_t(time: float):
            # Prepare some state
            # qml.Hadamard(0)
            circuit_VQE(theta, range(self.qchem_qubits))
            # Evolve according to H
            qml.QDrift(self.qchem_hamiltonian, time=time, n=10, seed=10)

            return qml.expval(self.qchem_hamiltonian)

        self.qdrift_t = qdrift_t

    def simulate_qdrift(self, t_end: float) -> float:
        return self.qdrift_t(t_end)

    def simulate_mrsqk(self, t_end: float, n_steps: int = 1) -> np.array:
        # Controlled time-evolution of qu_op
        c_trott = trotterize(
            self.qu_op, time=t_end, control=self.c_q,
            n_trotter_steps=n_steps
            # n_trotter_steps=int(np.ceil(t_end/0.2))
        )

        # Calculate MRSQK
        sab = np.zeros((self.n_krylov, self.n_krylov), dtype=complex)
        # np.all()
        # hab = np.zeros((self.n_krylov, self.n_krylov), dtype=complex)
        fhab = np.zeros((self.n_krylov, self.n_krylov), dtype=complex)

        resources = {}

        for a, b in product(range(self.n_krylov), range(self.n_krylov)):
            # Only need to do the upper triangle, 
            # and the rest is the conjugate by symmetry
            if a<b:
                continue
            
            # Generate Ua and Ub unitaries
            ua = (
                self.reference_states[a % 2] + c_trott * (a // 2)
                if a > 1
                else self.reference_states[a % 2]
            )
            ub = (
                self.reference_states[b % 2] + c_trott * (b // 2)
                if b > 1
                else self.reference_states[b % 2]
            )

            # Build circuit from Figure 2 for off-diagonal overlap
            hab_circuit = (
                Circuit([Gate("H", self.c_q)])
                + ua
                + Circuit([Gate("X", self.c_q)])
                + ub
            )

            sab[a, b] = (
                self.backend.get_expectation_value(self.zeroone, hab_circuit) / 2
            )
            sab[b, a] = sab[a, b].conj()

            # Hamiltonian matrix element for f(H) = e^{-i H \tau}
            fhab[a, b] = (
                self.backend.get_expectation_value(
                    self.zeroone, hab_circuit + c_trott.inverse()
                )
                / 2
            )
            # hab_circuit.
            resources = {**resources, **hab_circuit.counts_n_qubit, 'width': hab_circuit.width, 'depth': hab_circuit.depth}

        print(f"Circuit resources:\n{resources}")

        # e, v = scipy.linalg.eigh(hab, sab)
        # print(f"The HV=SVE energies are {np.real_if_close(np.round(e, 3))}")
        e = eigvals(fhab, sab)
        # print(f"The f(H)V=SVf(E) energies are {np.arctan2(np.imag(e), np.real(e))/tau}")
        return np.arctan2(np.imag(e), np.real(e)) / t_end



class MoleculeSimulator:
    def __init__(self, mol: SecondQuantizedMolecule) -> None:
        self.mol = mol
        self.h_sim = HamiltonianSimulator(mol)

    def ground_state_vqe(self) -> float:
        vqe_solver = VQESolver({"molecule": self.mol, "ansatz": BuiltInAnsatze.UCCSD})
        vqe_solver.build()
        vqe_energy = vqe_solver.simulate()
        print(vqe_solver.get_resources())
        print(f'Ground state \t VQE energy = {vqe_energy}')

        return vqe_energy
    
    def ground_state_classical(self) -> float:
        ccsd_solver = CCSDSolver(self.mol)
        return ccsd_solver.simulate()

    def excited_states_sa_vqe(self):
        vqe_options = {"molecule": self.mol, "ref_states": [[1,1,0,0], [1,0,0,1], [0,0,1,1]],
                    "weights": [1, 1, 1], "penalty_terms": {"S^2": [2, 0]},
                    "qubit_mapping": "jw", "ansatz": BuiltInAnsatze.UpCCGSD,
                    }
        vqe_solver = SA_VQESolver(vqe_options)
        vqe_solver.build()
        energy = vqe_solver.simulate()
        for i, energy in enumerate(vqe_solver.state_energies):
            print(f"Singlet State {i} has energy {energy}")

                # Generate individual statevectors
        ref_svs = list()
        for circuit in vqe_solver.reference_circuits:
            _, sv = vqe_solver.backend.simulate(circuit, return_statevector=True)
            ref_svs.append(sv)

        # Generate Equation (2) using equation (4) and (5) of arXiv:1901.01234
        h_theta_theta = np.zeros((3,3))
        for i, sv1 in enumerate(ref_svs):
            for j, sv2 in enumerate(ref_svs):
                if i != j:
                    sv_plus = (sv1 + sv2)/np.sqrt(2)
                    sv_minus = (sv1 - sv2)/np.sqrt(2)
                    exp_plus = vqe_solver.backend.get_expectation_value(vqe_solver.qubit_hamiltonian, vqe_solver.optimal_circuit, initial_statevector=sv_plus)
                    exp_minus = vqe_solver.backend.get_expectation_value(vqe_solver.qubit_hamiltonian, vqe_solver.optimal_circuit, initial_statevector=sv_minus)
                    h_theta_theta[i, j] = (exp_plus-exp_minus)/2
                else:
                    h_theta_theta[i, j] = vqe_solver.state_energies[i]

        e, _ = np.linalg.eigh(h_theta_theta)
        # print(e)
        for i, energy in enumerate(e):
            print(f"Singlet State {i} \t MC-VQE energy = {energy}")
        
        return e

        # algorithm_resources["sa_vqe"] = vqe_solver.get_resources()


    def excited_states_vqe_deflation(self, k: int) -> np.array:
        vqe_solver = VQESolver({"molecule": self.mol, "ansatz": BuiltInAnsatze.UCCSD})
        vqe_solver.build()
        vqe_energy = vqe_solver.simulate()
        energies = [vqe_energy]
        print(vqe_solver.get_resources())
        print(f"Ground state \t VQE energy = {vqe_energy}")

        # Add initial VQE optimal circuit to the deflation circuits list
        deflation_circuits = [vqe_solver.optimal_circuit.copy()]
        
        # Calculate first and second excited states by adding optimal circuits to deflation_circuits
        for i in range(1,k):
            vqe_solver = VQESolver({"molecule": self.mol, "ansatz": BuiltInAnsatze.UpCCGSD, 
                        "deflation_circuits": deflation_circuits, "deflation_coeff": 0.4})
            vqe_solver.build()
            vqe_energy = vqe_solver.simulate()
            print(vqe_solver.get_resources())
            print(f"Excited state #{i} \t VQE energy = {vqe_energy}")
            energies.append(vqe_energy)
        
        return np.array(energies)
    
    def hamiltonian_t_mrsqk(self, t: float) -> np.array:
        return self.h_sim.simulate_mrsqk(t)
    
    def hamiltonian_t_qdrift(self, t: float) -> float:
        return self.h_sim.simulate_qdrift(t)

In [57]:
mol_LiH = SecondQuantizedMolecule(
    [("Li", (0.0, 0.0, 0.0)), ("H", (0.0, 0.0, 1.547))],
    q=0, spin=0, basis="sto-3g",
)

sim = MoleculeSimulator(mol_LiH)

Qubit count: 11, control qubit: 10


In [51]:
mol_LiH.xyz

[('Li', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 1.547))]

In [52]:
ccsd_solver = CCSDSolver(mol_LiH)
energy_ccsd = ccsd_solver.simulate()
energy_ccsd

-7.882537790975925

In [53]:
sim.ground_state_vqe()

{'qubit_hamiltonian_terms': 276, 'circuit_width': 10, 'circuit_depth': 1805, 'circuit_2qubit_gates': 1376, 'circuit_var_gates': 144, 'vqe_variational_parameters': 14}
Ground state 	 VQE energy = -7.882536730639694


-7.882536730639694

In [54]:
sim.simulate_excited_states(3)

AttributeError: 'MoleculeSimulator' object has no attribute 'simulate_excited_states'

In [58]:
sim.hamiltonian_t_mrsqk(0.001)[:2]

{'H': 1329, 'CX': 4, 'X': 1, 'PHASE': 2, 'CRZ': 550, 'RX': 1328, 'CNOT': 4724}


tensor([-7.86311963, -7.70734698], requires_grad=True)

In [13]:
from pyscf import mcscf

myhf = mol_LiH.mean_field
ncore = {"A1g": 1, "A1u": 1}
ncas = {"A1g": 1, "A1u": 1}

# print("Calculation for A1g symmetry")
mc = mcscf.CASCI(myhf, 2, (1, 1))
# # mo = mc.sort_mo_by_irrep(cas_irrep_nocc=ncas, cas_irrep_ncore=ncore)
mo = mc.sort_mo_by_irrep(cas_irrep_nocc=ncas, cas_irrep_ncore=ncore)
# mc.fcisolver.wfnsym = "A1g"
# mc.fcisolver.nroots = 2
# emc_A1g = mc.casci(mo)[0]

print("\n Calculation for A1u symmetry")
mc = mcscf.CASCI(myhf, 2, (1, 1))
mc.fcisolver.wfnsym = "A1u"
mc.fcisolver.nroots = 2
emc_A1u = mc.casci(mo)[0] 

AttributeError: 'CASCI' object has no attribute 'sort_mo_by_irrep'

In [37]:

from pyscf import gto, scf, cc


mol = gto.Mole()
mol.verbose = 5
mol.unit = 'A'
mol.atom = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.547'
# mol.atom = 'O 0 0 0; O 0 0 1.2'
# mol.basis = 'ccpvdz'
# mol.basis = 'STO-G3'
mol.build()

mf = scf.RHF(mol).run()

mc = mcscf.CASCI(mf, 4,4)
mc.state_specific_(2)
excited_grad = mc.nuc_grad_method().as_scanner()
mol1 = excited_grad.optimizer().kernel()
print(mol1)

# mf = scf.RHF(mol)
# mf.verbose = 7
# mf.scf()

# mycc = cc.RCCSD(mf)
# mycc.verbose = 7
# mycc.ccsd()

# eip,cip = mycc.ipccsd(nroots=1)
# print(eip,cip)
# eea,cea = mycc.eaccsd(nroots=1)
# print(eea,cea)

# eee,cee = mycc.eeccsd(nroots=1)
# print(eee,cee)

# # S->S excitation
# eS = mycc.eomee_ccsd_singlet(nroots=1)[0]
# print(eS)
# # S->T excitation
# eT = mycc.eomee_ccsd_triplet(nroots=1)[0]
# print(eT)


# # # Triplet

# mol.spin = 2
# mol.build()

# mf = scf.UHF(mol)
# mf.verbose = 7
# mf.scf()

# mycc = cc.UCCSD(mf)
# mycc.verbose = 7
# mycc.ccsd()

# eip,cip = mycc.ipccsd(nroots=1)
# eea,cea = mycc.eaccsd(nroots=1)

# # EOM-GCCSD
# mf = mf.to_ghf()
# mycc = mf.CCSD()
# ecc, t1, t2 = mycc.kernel()
# e,v = mycc.ipccsd(nroots=6)
# e,v = mycc.eaccsd(nroots=6)
# e,v = mycc.eeccsd(nroots=6)

System: uname_result(system='Darwin', node='Rohans-MacBook-Pro.local', release='23.6.0', version='Darwin Kernel Version 23.6.0: Wed Jul 31 20:50:00 PDT 2024; root:xnu-10063.141.1.700.5~1/RELEASE_ARM64_T6031', machine='arm64')  Threads 1
Python 3.12.5 (main, Aug 16 2024, 22:03:57) [Clang 15.0.0 (clang-1500.3.9.4)]
numpy 1.26.4  scipy 1.14.1  h5py 3.11.0
Date: Fri Sep 27 12:37:14 2024
PySCF version 2.6.2
PySCF path  /Users/rohan/.pyenv/versions/3.12.5/envs/nyc_haq/lib/python3.12/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 5
[INPUT] max_memory = 4000 
[INPUT] num. atoms = 2
[INPUT] num. electrons = 4
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = A
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 Li     0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00000000000

ModuleNotFoundError: No module named 'geometric'

In [20]:
print(eip,cip)
print(eea,cea)
print(eee,cee)

# S->S excitation
print(eS)
# S->T excitation
print(eT)


0.27184072197393533 [-4.04808438e-04 -9.60200575e-01 -2.22119054e-03  8.62248804e-19
  6.69307305e-19  1.92996975e-03  6.73500836e-04 -3.52187884e-19
 -7.79666701e-19 -1.69570823e-03  6.16006795e-03 -1.35295725e-18
 -8.12069294e-19  2.04535191e-03  1.68865661e-01 -5.71452437e-17
  1.31174849e-18 -2.22362235e-01]
0.07683374064938706 [-9.95428722e-01  2.43400345e-16  2.13258892e-16  7.30406794e-03
  1.03949629e-03  1.29508180e-18  2.84582958e-18  4.12830244e-03
  1.09185403e-18  4.42595471e-03 -8.05242902e-20 -1.75234063e-18
  1.00733957e-18 -4.84412302e-20  4.42595471e-03 -1.44686022e-18
  1.82311933e-03 -1.21178863e-18 -8.55098115e-19  1.45337992e-03
  1.26779358e-02  2.24825314e-17 -1.01296525e-16 -3.63536137e-03
  1.72282140e-17  6.01274282e-02  9.19652097e-19 -1.42259892e-17
  4.68996474e-18 -1.62045558e-19  6.01274282e-02 -7.75326172e-18
  2.28827402e-02 -2.03485574e-17 -9.28748427e-18 -3.28236381e-02]
0.11903923021473461 [-5.13930729e-03  8.14596875e-19  2.41780932e-19 -1.42293649